In [1]:
!nvidia-smi

Thu Jul 22 11:13:31 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 457.09       Driver Version: 457.09       CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 166... WDDM  | 00000000:26:00.0  On |                  N/A |
|  0%   48C    P8    10W / 140W |    537MiB /  6144MiB |      8%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [21]:
import torch
from torch import nn

In [5]:
def try_gpu(i=0):
    # 如果存在，返回gpu(i)，否则返回cpu()
    if torch.cuda.device_count() >= i+1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():
    # 返回所有可用的gpu
    devices = [
        torch.device(f'cuda:{i}') for i in range(torch.cuda.device_count())
    ]
    return devices if devices else [torch.device('cpu')]

In [6]:
try_gpu()

device(type='cuda', index=0)

In [7]:
try_gpu(10)

device(type='cpu')

In [8]:
try_all_gpus()

[device(type='cuda', index=0)]

在GPU上创建tensor

In [9]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

In [10]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [11]:
Y = torch.rand(2, 3, device=try_gpu(10))
Y

tensor([[0.6305, 0.2264, 0.2176],
        [0.1224, 0.0300, 0.4697]])

In [15]:
Z = Y.cuda(0)  # 把CPU上的Y复制到GPU0上
print(Y)
print(Z)

tensor([[0.6305, 0.2264, 0.2176],
        [0.1224, 0.0300, 0.4697]])
tensor([[0.6305, 0.2264, 0.2176],
        [0.1224, 0.0300, 0.4697]], device='cuda:0')


In [16]:
X + Z # 同在GPU0上的数据

tensor([[1.6305, 1.2264, 1.2176],
        [1.1224, 1.0300, 1.4697]], device='cuda:0')

In [18]:
X + Y # 跨设备，报错

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

In [19]:
Z.cuda(0) is Z

True

神经网络与GPU

In [22]:
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())
net(X)

tensor([[1.1440],
        [1.1440]], device='cuda:0', grad_fn=<AddmmBackward>)

In [23]:
net[0].weight.data.device

device(type='cuda', index=0)